In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tqdm, re, pandas as pd, random, pprint

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
# Find the common users
regularBefore= RegB[RegB['user.id_str'].isin(RegA['user.id_str'])].copy()
regularAfter= RegA[RegA['user.id_str'].isin(RegB['user.id_str'])].copy()
# Set the type of user
regularBefore['typeOfUser']= 'regularBefore'
regularAfter['typeOfUser']= 'regularAfter'

In [ ]:
ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
# Find he common users
restoredBefore= ResB[ResB['user.id_str'].isin(ResA['user.id_str'])==True].copy()
restoredAfter= ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()
# Set the type of user
restoredBefore['typeOfUser']= 'restoredBefore'
restoredAfter['typeOfUser']= 'restoredAfter'

In [ ]:
restoredBefore = restoredBefore[['user.id_str', 'created_at', 'id_str']].copy()
restoredAfter = restoredAfter[['user.id_str', 'created_at', 'id_str']].copy()
regularBefore = regularBefore[['user.id_str', 'created_at', 'id_str']].copy()
regularAfter = regularAfter[['user.id_str', 'created_at', 'id_str']].copy()

In [ ]:
pklDict= {
    'restoredBefore': restoredBefore,
    'restoredAfter': restoredAfter,
    'regularBefore': regularBefore,
    'regularAfter': regularAfter,
}
ResA= None
ResB= None
RegA= None
RegB= None
regularBefore= None
regularAfter= None
restoredBefore= None
restoredAfter= None

In [ ]:
pklDict['restoredBefore']['created_at']= pd.to_datetime(pklDict['restoredBefore']['created_at']).dt.tz_convert('Asia/Kolkata')
pklDict['restoredAfter']['created_at']= pd.to_datetime(pklDict['restoredAfter']['created_at']).dt.tz_convert('Asia/Kolkata')
pklDict['regularBefore']['created_at']= pd.to_datetime(pklDict['regularBefore']['created_at']).dt.tz_convert('Asia/Kolkata')
pklDict['regularAfter']['created_at']= pd.to_datetime(pklDict['regularAfter']['created_at']).dt.tz_convert('Asia/Kolkata')

In [ ]:
temp= pklDict['regularAfter']
pklDict['regularAfter']= temp[temp['created_at'].dt.year==2019]

temp= pklDict['regularBefore']
pklDict['regularBefore']= temp[temp['created_at'].dt.year==2019]

temp= pklDict['restoredAfter']
pklDict['restoredAfter']= temp[temp['created_at'].dt.year==2019]

temp= pklDict['restoredBefore']
pklDict['restoredBefore']= temp[temp['created_at'].dt.year==2019]

temp= None

In [ ]:
pklDict['regularBefore']= pklDict['regularBefore'][pklDict['regularBefore']['user.id_str'].isin(pklDict['regularAfter']['user.id_str'])].copy()
pklDict['regularAfter']= pklDict['regularAfter'][pklDict['regularAfter']['user.id_str'].isin(pklDict['regularBefore']['user.id_str'])].copy()
pklDict['restoredBefore']= pklDict['restoredBefore'][pklDict['restoredBefore']['user.id_str'].isin(pklDict['restoredAfter']['user.id_str'])].copy()
pklDict['restoredAfter']= pklDict['restoredAfter'][pklDict['restoredAfter']['user.id_str'].isin(pklDict['restoredBefore']['user.id_str'])].copy()

In [ ]:
# import pickle
# # open a file, where you ant to store the data
# file = open('/suspended_restored_account/code/pklDict', 'wb')

# # dump information to that file
# pickle.dump(pklDict, file)

# # close the file
# file.close()

In [ ]:
regular_timeline= pd.concat([pklDict['regularBefore'], pklDict['regularAfter']], ignore_index= True)
restored_timeline= pd.concat([pklDict['restoredBefore'], pklDict['restoredAfter']], ignore_index= True)
pklDict= None

In [ ]:
# Define day wise groups
regular_groups= regular_timeline.groupby(regular_timeline['created_at'].dt.date)
restored_groups= restored_timeline.groupby(restored_timeline['created_at'].dt.date)

In [ ]:
# Regular

uniqueRegularCount= len(regular_timeline['user.id_str'].unique())
print(f'Unique Users {uniqueRegularCount}')

x_0, y_0= [], []
for name, group in regular_groups:
    if len(group.index)>0:
        x_0.append(name)
        print(f'Tweets on {name} were {len(group.index)}')
        y_0.append(len(group.index)/uniqueRegularCount)

In [ ]:
# Restored

uniqueRestoredCount= len(restored_timeline['user.id_str'].unique())
print(f'Unique Users {uniqueRestoredCount}')

x_1, y_1= [], []
for name, group in restored_groups:
    if len(group.index)>0:
        x_1.append(name)
        print(f'Tweets on {name} were {len(group.index)}')
        y_1.append(len(group.index)/uniqueRestoredCount)

In [ ]:
fig= plt.figure()
fig.set_size_inches(12, 6)
ax= fig.add_subplot(1, 1, 1)
axes.set_xlabel("Date", fontsize= 27)
axes.set_ylabel("No. of tweets", fontsize= 27)
# plt.xlabel('Date')
# plt.ylabel('Number of Tweets')
ax.tick_params(axis= 'x', labelsize= 20)
ax.tick_params(axis= 'y', labelsize= 20)
plt.plot(x_0, y_0, c='0.5', label='Regular')
plt.plot(x_1, y_1, c='0.2', label='Restored')
plt.legend()
plt.show()